Import used packages

In [1]:
import cv2
import os
import numpy as np
from algorithms import *

Define image path

In [2]:
image_path = './assets/torino.jpg'
file_extension = os.path.splitext(image_path)[1]
image_type = file_extension[1:].lower()

Load RGB image

In [3]:
# Load the image in color format
img = cv2.imread(image_path, cv2.IMREAD_COLOR)

Change image to matrix

In [4]:
matrix = np.float32(img)

# Print the matrix shape
print("Matrix shape:", matrix.shape)

Matrix shape: (4000, 6000, 3)


In [7]:
original_size = save_image(matrix, image_type, onlySize=True)
print(f'{original_size} MB')

4.2088 MB


Split the image into three separate color channels

In [6]:
# Slice the third dimension of the array

b = matrix[:,:,0]
g = matrix[:,:,1]
r = matrix[:,:,2]

Apply SVD to the channels